In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import pickle
import math

import tensorflow as tf 
from tensorflow import keras
from keras import backend as K
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
data_filename = '/Users/brad/Desktop/CS6620/Project/Data/dataVersion1.npy' 
truth_filename = '/Users/brad/Desktop/CS6620/Project/Data/truthVersion1.npy' 
x = np.load(data_filename)
y = np.load(truth_filename)
print(x.shape)
print(y.shape)

(5259, 34)
(5259,)


In [3]:
x[0].shape

(34,)

In [8]:
train_length = int(x.shape[0] * 0.85)
x_train = x[0 : train_length].reshape(-1, 34)
y_train = y[0 : train_length].reshape(-1, 1)
x_test = x[train_length : ].reshape(-1, 34)
y_test = y[train_length : ].reshape(-1, 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4470, 34)
(4470, 1)
(789, 34)
(789, 1)


In [4]:
plt.figure()
plt.hist(y_train, edgecolor='black', linewidth=1)
plt.title("Training data")
plt.xlabel("Generation")
plt.ylabel("Occurences")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
plt.figure()
plt.hist(y_test, edgecolor='black', linewidth=1)
plt.title("Validation data")
plt.xlabel("Generation")
plt.ylabel("Occurences")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# computes an R^2 (goodness of fit) approximation 
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [29]:
model = Sequential()
model.add(Dense(128, input_dim=34, activation='relu')) # for ann models 
# model.add(LSTM(128, input_shape=(34,1), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
# optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=0.05, decay=1e-5)
# compile
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[coeff_determination])

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(34,1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
# optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-5)
# compile
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[coeff_determination])

In [10]:
def plot_loss(history):
  plt.figure()
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.grid(True)
  plt.show()
  plt.figure()
  plt.plot(history.history['coeff_determination'], label='R^2')
  plt.plot(history.history['val_coeff_determination'], label='validation R^2')
  plt.xlabel('Epoch')
  plt.ylabel('R^2')
  plt.legend()
  plt.show()
  

In [30]:
history= model.fit(x=x_train, y=y_train, batch_size=15, epochs=250, validation_data=(x_test, y_test))
plot_loss(history)

34/250
298/298 [==============================] - 0s 706us/step - loss: 0.0721 - coeff_determination: -1.3128 - val_loss: 0.0163 - val_coeff_determination: -3.8456
Epoch 135/250
298/298 [==============================] - 0s 713us/step - loss: 0.0737 - coeff_determination: -1.3114 - val_loss: 0.0163 - val_coeff_determination: -3.8456
Epoch 136/250
298/298 [==============================] - 0s 716us/step - loss: 0.0733 - coeff_determination: -1.3482 - val_loss: 0.0163 - val_coeff_determination: -3.8456
Epoch 137/250
298/298 [==============================] - 0s 719us/step - loss: 0.0695 - coeff_determination: -1.2990 - val_loss: 0.0163 - val_coeff_determination: -3.8456
Epoch 138/250
298/298 [==============================] - 0s 707us/step - loss: 0.0715 - coeff_determination: -1.3870 - val_loss: 0.0163 - val_coeff_determination: -3.8456
Epoch 139/250
298/298 [==============================] - 0s 730us/step - loss: 0.0746 - coeff_determination: -1.3843 - val_loss: 0.0163 - val_coeff_dete

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot_loss(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
preds = model.predict(x_test)

In [22]:
print(preds[4][0])
print(y_test[40][0])


0.11360767
0.030004727349974485


In [23]:
def computeAverageError(pred, y):
    err = []
    for i in range(len(pred)):
        err.append(abs((y[i] - pred[i][0])/y[i][0]))

    return sum(err)/ len(err)

In [24]:
computeAverageError(preds, y_test)

array([1.25671689])

In [25]:
preds2 = model.predict(x_train)
computeAverageError(preds2, y_train)[0]

1.2078017378450374

In [13]:
plt.figure()
plt.scatter(x=np.arange(len(y_train)), y=y_train)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.figure()
plt.scatter(x=np.arange(len(y_train)), y=preds2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
import torch
from torch import nn
import numpy as np